In [5]:
%matplotlib inline 
from numpy import *
import numpy as np 
import scipy
from pylab import* 
import pylab as pylab



Les données

In [58]:
nb_stations = 5
nb_velos = 50

# Table des temps moyens de trajets (convertis en heures)
Tps_trajets = 1/60 * np.array([[0, 3, 5, 7, 7],
                               [2, 0, 2, 5, 5],
                               [4, 2, 0, 3, 3], 
                               [8, 6, 4, 0, 2], 
                               [7, 7, 5, 2, 0]])

# Table des taux de départs par heures
Taux_dep = [2.8, 3.7, 5.5, 3.5, 4.6]

# Matrice de routage
P = np.array([[0, 0.22, 0.32, 0.2, 0.25],
              [0.17, 0, 0.34, 0.21, 0.28],
              [0.19, 0.26, 0, 0.24, 0.31],
              [0.17, 0.22, 0.33, 0, 0.28],
              [0.18, 0.24, 0.35, 0.23, 0]])


Les colonnies : une pour chaque stations + une pour chaque trajet possible = nb_stations + nb_stations*(nb_stations-1) = nb_stations

On note :

- n_ii : nombre de vélos dans la station n°i
- n_ij : nombre de vélos effectuant le trajet de la station n°i à station n°j

Transitions possibles : 

- une arrivée de vélo depuis la station n°i vers la station n°j 
  
    n --> n - e_ij + e_jj avec taux $\frac{1}{T_{trajets}\{i,j\}} n_{ij} $ (pour i différent de j)

- un départ de vélo depuis la station n°i vers la station n°j

    n --> n - e_ii + e_ij avec taux  $P_{ij} * Taux de départ_i * n_{ii}$ 


In [59]:
# Matrice des taux de type arrivée (1/Tps_trajets_ij)
Ta = 1/Tps_trajets
np.fill_diagonal(Ta, 0)


# Matrice des taux de type départ (P_ij * Taux de départ_i)
Td = np.multiply(Taux_dep, P.T).T

print(Td)



[[0.    0.616 0.896 0.56  0.7  ]
 [0.629 0.    1.258 0.777 1.036]
 [1.045 1.43  0.    1.32  1.705]
 [0.595 0.77  1.155 0.    0.98 ]
 [0.828 1.104 1.61  1.058 0.   ]]


/var/folders/hq/nz6rv6b136j0sk_knhths__w0000gn/T/ipykernel_53475/4218666887.py:2: RuntimeWarning: divide by zero encountered in true_divide
  Ta = 1/Tps_trajets


In [71]:
Ntest = np.full((nb_stations,nb_stations), int(nb_velos/(nb_stations**2)))

def transition(N):
    #
    A_arrivee = np.multiply(Ta, N).flatten()
    A_depart = (np.multiply(np.diag(N), Td.T).T).flatten()

    A_tot = np.concatenate([A_arrivee, A_depart])

    r = np.cumsum(A_tot)/np.sum(A_tot)

    u = 0.06 #rand()
    w = 0

    while u > r[w]:
        w+=1

    print(u)
    print(r)
    print(w)
    # Transition de type arrivée de station n°i vers n°j
    if w < nb_stations**2 :
        i, j = w//nb_stations, w%nb_stations
        N[i,j] -= 1
        N[j,j] += 1
            
    # Transition de type départ de station n°i vers n°j
    else : 
        w -= nb_stations**2
        i, j = w//nb_stations, w%nb_stations
        N[i,i] -= 1
        N[i,j] += 1
        
    return

print(transition(Ntest))



# Paramètre de la loi expo du temps de séjour en un état n
def A(N):
    A_N = 0
    for i in range(nb_stations):
        for j in range(i+1, nb_stations):
            A_N += N[i,j]/Tps_trajets[i,j] + N[j,i]/Tps_trajets[j,i] + P[i,j] * N[i,i]*P[i,j]*Taux_dep[i] + N[j,j]*P[j,i]*Taux_dep[j]
    return A_N

0.06
[0.         0.05557752 0.08892404 0.11274297 0.13656191 0.21992819
 0.21992819 0.30329448 0.33664099 0.3699875  0.41167064 0.49503693
 0.49503693 0.55061445 0.60619197 0.62703354 0.6548223  0.69650544
 0.69650544 0.77987173 0.80369067 0.8275096  0.86085612 0.9442224
 0.9442224  0.9442224  0.94593419 0.94842406 0.94998023 0.95192544
 0.95367336 0.95367336 0.95716918 0.95932837 0.96220729 0.96511121
 0.969085   0.969085   0.97275312 0.9774911  0.97914453 0.98128427
 0.98449387 0.98449387 0.98721717 0.98951808 0.99258596 0.99705995
 1.         1.        ]
2
('Arrivée', 0, 2)


In [39]:
# état initial (réparti uniformément sur les colonnies)
N = np.full((nb_stations,nb_stations), int(nb_velos/(nb_stations**2)))

#N = randint(0,int(nb_velos/(nb_stations**2)) + 1,(nb_stations, nb_stations))
#N[nb_stations-1, nb_stations-1] = nb_stations - sum(N) - N[nb_stations-1, nb_stations-1] 

print(A(N))

705.4393485714286


In [ ]:
horizon=20
temps=0

while temps <= horizon :
    duree = -1/A(N) * log(rand())
    # prochaine transition ?
    



In [13]:
rho=0.5
horizon=10000
temps=0
N=0
K=int(np.trunc(log(0.001)/log(rho)))+1
proba_stat=np.zeros(K+1)
nb_moyen_clients=0

def transitions(N): 
    if (N==0):
        return([rho,[1],[1]]) 
    else:
        return([rho+1,[N-1,N+1],[1/(rho+1),rho/(rho+1)]])

while(temps<= horizon): a = transitions(N)
#
#
print N,a
duree=np.random.exponential(1/a[0])
temps += duree
proba_stat[min(N,K)]+= duree
N=np.random.choice(a[1],p=a[2])
print N
proba_stat =proba_stat/horizon
pylab.clf()
pylab.bar(np.arange(K+1),proba_stat)
pylab.plot(np.arange(K+1),(1-rho)*rho**np.arange(K+1),color='red')
pylab.savefig('geometric')

0.5413503559189221
